In [10]:
#!pip install pyarrow
#!pip install tensorflow

In [1]:
import numpy as np
import pandas as pd
import pyarrow

In [2]:
df = pd.read_parquet('dataset.parquet', engine='pyarrow')

In [3]:
#Interpolating linearly for the missing frames
df = df.interpolate(method='linear', axis=0, limit_direction='both')

In [ ]:
df.to_parquet('prep_dataset.parquet', index=True)

In [4]:
display(df[1828:1831])

,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
34,P04,Abduction,Correct,Frontal_Low,1.133333,0.061257,-0.350373,0.344725,0.003512,0.575839,...,-0.151298,-0.025552,0.541597,-0.138189,-0.035060,1.027590,-0.148856,-0.032272,1.014960,-0.140562
35,P04,Abduction,Correct,Frontal_Low,1.166667,0.022444,0.197917,0.276030,0.032612,-0.013347,...,-0.207359,-0.015390,0.044670,-0.139330,0.141457,0.496583,-0.183538,0.131540,0.487554,-0.184331
36,P04,Abduction,Correct,Frontal_Low,1.200000,-0.016368,0.746207,0.207336,0.061712,-0.602534,...,-0.263419,-0.005228,-0.452257,-0.140470,0.317974,-0.034424,-0.218219,0.295352,-0.039851,-0.228100


In [ ]:
params_columns = ['Participant', 'Exercise', 'Set', 'Camera']
df['SequenceLength'] = df.groupby(params_columns)['time(s)'].transform('count')
max_sequence_length = df['SequenceLength'].max()

padded_sequences = []
for sequence_id, group in df.groupby(params_columns):
    sequence_data = group.iloc[:, 4:-1].values
    pad_width = ((0, max_sequence_length - len(sequence_data)), (0, 0))
    padded_sequence = np.pad(sequence_data, pad_width, mode='constant', constant_values=0)
    padded_sequences.append(padded_sequence)

In [ ]:
padded_df = pd.DataFrame(np.concatenate(padded_sequences, axis=0), columns=df.columns[4:-1])
padded_df[params_columns] = np.repeat(df[params_columns].drop_duplicates().values, max_sequence_length, axis=0)
padded_df = padded_df[['Participant', 'Exercise', 'Set', 'Camera'] + list(padded_df.columns[:-4])]

In [ ]:
print(max_sequence_length)

In [ ]:
#padded_df.to_parquet('padded_dataset.parquet', index=True)